In [ ]:
# This code compute preliminary statistical analyses on RS CLOSED DATA
# 1) Descriptive statistics
# 2) Linear mixed model on every region with group and resting state block as fixed effects, and subject ID as random effect
# 3) Clustering of ASD subject based on general IQ
# 2) Linear mixed model on every region with IQ-based subgroups and resting state block as fixed effects, and subject ID as random effect


# Import libraries and set directories

import os
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt

# environment name: explore_cmifoof

# Set paths
main = ''



data = pd.read_csv(os.path.join(main,"rs_closed_alpha_regions_AllBlocks.csv"))

data.head()

In [3]:
# Set parameter of interest

parameter = "alpha_amplitude"


In [ ]:
# Load infos about EEG montage and display regions channels

from IPython.display import Image, display

# Load and display an image
image_path = os.path.join(main, "EEG_regions_electrodes.png") # Update this to the path of your image file
display(Image(filename=image_path))


# Read channel dictonaries, defining for each area the number of the electrodes
channel_dict = {
        "frontal_left": [23, 24, 27, 28],
        "frontal_midline": [5, 11, 12, 16],
        "frontal_right": [3, 117, 123, 124],
        "central_left": [ 35, 36, 41, 42],
        "central_midline": [7, 31, 80, 106],
        "central_right": [93, 103, 104, 110],
        "posterior_left": [51, 52, 59, 60],
        "posterior_midline": [62, 71, 72, 76],
        "posterior_right": [85, 91, 92, 97]
                 }



In [ ]:
# Compute descriptive statistics

asd_data = data[data['diagnosis']=="ASD"]
td_data = data[data['diagnosis']=="TD"]

for region in channel_dict.keys():

    # Get the name of each region
    region_column = f"{parameter}_{region}"
    
    asd_data_region = asd_data[region_column]
    td_data_region = td_data[region_column]

    # Print values
    print(f"Mean value for {region_column} in ASD is {asd_data[region_column].mean()}")
    print(f"Mean value for {region_column} in TD is {td_data[region_column].mean()}")
    print(f"Standard Deviation for {region_column} in ASD is {asd_data[region_column].std()}")
    print(f"Standard Deviation for {region_column} in TD is {td_data[region_column].std()}")

In [ ]:
# For every region:
# 1) Plot case-control differences in the parameter
# 2) Compute for every region the correlation between parameter value and FIQ in ASD group.
#    This is done to explore the hypothesized association between alpha amplitude and cognitive development in Autism.

for region in channel_dict.keys():

    region_column = f"{parameter}_{region}"

    # Plot boxplot
    data.boxplot(column=region_column, by='diagnosis')
    plt.show()

    # Plot scatter 
    plt.scatter(asd_data[region_column], asd_data['fiq'])
    plt.title('Correlation between alpha region amplitude and IQ in Autism')

    # Remove NA
    asd_data_clean = asd_data.dropna(subset=[region_column, 'fiq'])

    # Compute pearson correlation
    r_pears = stats.pearsonr(asd_data_clean[region_column], asd_data_clean['fiq'])
    print(r_pears)



In [ ]:
# For every region compute a mixed linear model with diagnosis and rs time point (1,2,3,4,5) as fixed effect and subject 
# as random effect
from statsmodels.formula.api import mixedlm

for region in channel_dict.keys():
  
    # Get region name
    region_column = f"{parameter}_{region}"
    filtered_data = data[data[region_column].notnull()] # Remove missing values

    # Generate formula and model
    model_formula = f"{region_column} ~ diagnosis * rs_block"
    model = mixedlm(model_formula, filtered_data, groups=filtered_data["subid"])

    # Fit the model
    result = model.fit()

    # Print the summary of the model
    print(region_column)
    print(result.summary())

In [ ]:
# Now, I want to explore full scale iq data distribution for the two groups.
# If ASD show a multiple-peak density distribution, I'm proceeding with subgrouping them based on IQ.

# Custom color palette
custom_palette = {'ASD': 'blue', 'TD': 'green'}

# Plot the histogram
plt.figure(figsize=(10, 6))
sns.histplot(data, x='fiq', hue='diagnosis', kde=True, bins=10, palette=custom_palette, element='step', stat='density')


# Adding labels and title
plt.xlabel('IQ')
plt.ylabel('Density')
plt.title('Histogram of IQ Scores by Group')
plt.legend(title='Diagnosis')  # LP: legend showing up empty - maybe issue with my package?

# Show the plot
plt.show()


In [ ]:
# ASD iq data show two peaks.
# Now, I'm perform k-means clustering (k-optimal = 2) to autism cluster subjects based on IQ
# LP: I am not very sure about clustering 1D data. You would really need to prove multiple peaks in the data distribution
# beforehand, and this was not super clear from the histogram above - otherwise you just split continuous data into arbitrary groups!
# But this is not Python, sorry if I'm wrong :)


from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Filter the data to include only the ASD group
asd_data = data[data['diagnosis'] == 'ASD']
asd_data = data[data['fiq'].notnull()] 


# Get all subjects IQ
fiq_scores = asd_data[['fiq']]

# Standardize the data
scaler = StandardScaler()
fiq_scores_scaled = scaler.fit_transform(fiq_scores)

# Determine the number of clusters using the elbow method
sum_of_squared_distances = []
K = range(1, 10)

for k in K:
    kmeans = KMeans(n_clusters=k)
    kmeans = kmeans.fit(fiq_scores_scaled)
    sum_of_squared_distances.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('Number of clusters')
plt.ylabel('Sum of squared distances')
plt.title('Elbow Method For Optimal k')
plt.show()

# From the elbow plot, choose an appropriate number of clusters (e.g., 3)
optimal_k = 4

# Perform K-Means clustering
kmeans = KMeans(n_clusters=optimal_k)
clusters = kmeans.fit_predict(fiq_scores_scaled)

# Add the cluster labels to the original data
asd_data['Cluster'] = clusters

# Visualize the clusters
# LP: since you're clustering 1D data, a better visualization would have been the histograms
# for each cluster.
plt.figure(figsize=(10, 6))
sns.scatterplot(x='fiq', y='Cluster', data=asd_data, hue='Cluster', palette='viridis', s=100)
plt.xlabel('fiq')
plt.ylabel('Cluster')
plt.title('Clusters of ASD Group Based on full scale IQ')
plt.legend(title='Cluster')
plt.show()



In [ ]:
# Show clusters and their FIQ pattern

plt.figure(figsize=(10, 6))
sns.boxplot(x='Cluster', y='fiq', data=asd_data, palette='husl')

asd_data['Cluster']

In [ ]:
# Give labels to clusters
asd_data['FIQ_subgroup'] = asd_data['Cluster'].map({0: 'HIGH', 1: 'LOW', 2: 'VERY_HIGH', 3:'VERY_LOW'})

# Add fake cluster label also to TD group
td_data['Cluster'] = 85 #arbitrary number to fill the column
td_data['FIQ_subgroup'] = "TD"

# Concatenate vertically ASD and TD dataframe (stack rows)
all_data_subgroups = pd.concat([asd_data, td_data], axis=0, ignore_index=True)


In [14]:
# Initialize function to compute pairwise comparisons

from scipy import stats



def pairwise_comparisons(coefficients, std_errors, labels):
    num_groups = len(coefficients)
    comp_results = []
    for i in range(num_groups):
        for j in range(i + 1, num_groups):
            # Difference in coefficients
            effect_diff = coefficients[i] - coefficients[j]
            # Standard error of the difference
            se_diff = np.sqrt(std_errors[i]**2 + std_errors[j]**2)
            # Z-score
            z = effect_diff / se_diff
            # Two-tailed p-value
            p_value = 2 * (1 - stats.norm.cdf(np.abs(z)))
            comp_results.append((labels[i], labels[j], effect_diff, se_diff, z, p_value))
    return comp_results




In [ ]:
# For every region compute a mixed linear model with FIQ subtype and rs time point (1,2,3,4,5) as fixed effect and subject 
# as random effect
# LP: numpy import was missing! after you're done with a notebook always try to restart it and try to run all cells to see if everything works
import numpy as np 

# LP: am I right in assuming you are comparing an unsplit population (TD) with a split population (ASD)?
# I do not think this is sound if true!
for region in channel_dict.keys():

    
    region_column = f"{parameter}_{region}"
    filtered_data = all_data_subgroups[all_data_subgroups[region_column].notnull()]

    model_formula = f"{region_column} ~ FIQ_subgroup * rs_block"
    model = mixedlm(model_formula, filtered_data, groups=filtered_data["subid"])

    # Fit the model
    result = model.fit()

    # Print the summary of the model
    print(region_column)
    print(result.summary())

    #
    coefficients = result.fe_params[1:5]
    std_errors = result.bse_fe[1:5]

    labels = ['LOW', 'TD', 'VERY_HIGH', 'VERY_LOW']

    # Perform pairwise comparisons
    comparisons = pairwise_comparisons(coefficients, std_errors, labels)

    # Print results
    for comp in comparisons:
        print(f"Comparison: {comp[0]} vs {comp[1]}")
        print(f"  Effect Difference: {comp[2]:.3f}")
        print(f"  SE Difference: {comp[3]:.3f}")
        print(f"  Z-Score: {comp[4]:.3f}")
        print(f"  P-Value: {comp[5]:.3f}")
        print()